# Clustering Crypto

In [109]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [110]:
# Load the crypto_data.csv dataset.
file_path="data/crypto_data.csv"
crypto_df=pd.read_csv(file_path)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [111]:
# Keep all the cryptocurrencies that are being traded.
# Check what values are in the IsTrading Column
crypto_df["IsTrading"].value_counts()

True     1144
False     108
Name: IsTrading, dtype: int64

In [112]:
# What type of data is IsTrading column????
crypto_df["IsTrading"].dtype

dtype('bool')

In [113]:
# Filter to only True is IsTrading after treating true as a boolean!
crypto_df=crypto_df[crypto_df["IsTrading"]==True]

In [114]:
# Verify that data was filtered to True
crypto_df["IsTrading"].value_counts()

True    1144
Name: IsTrading, dtype: int64

In [115]:
# Review Data frame and confirm that it matches the number of true rows
print(crypto_df.head())
print(crypto_df.shape)

  Unnamed: 0   CoinName Algorithm  IsTrading ProofType  TotalCoinsMined  \
0         42    42 Coin    Scrypt       True   PoW/PoS     4.199995e+01   
1        365    365Coin       X11       True   PoW/PoS              NaN   
2        404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09   
3        611  SixEleven   SHA-256       True       PoW              NaN   
4        808        808   SHA-256       True   PoW/PoS     0.000000e+00   

  TotalCoinSupply  
0              42  
1      2300000000  
2       532000000  
3          611000  
4               0  
(1144, 7)


In [116]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns="IsTrading",inplace=True)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [117]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df["Algorithm"].value_counts()

Scrypt                   394
X11                      182
SHA-256                  121
X13                       54
PoS                       42
                        ... 
VeChainThor Authority      1
Ouroboros                  1
POS 2.0                    1
Proof-of-BibleHash         1
TRC10                      1
Name: Algorithm, Length: 89, dtype: int64

In [118]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace=True)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [119]:
# Keep the rows where coins are mined.
crypto_df=crypto_df[crypto_df["TotalCoinsMined"]>0]
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [120]:
# Make a copy of the Unamed: 0 for later use
names = crypto_df["Unnamed: 0"]

In [121]:
crypto_df.set_index("Unnamed: 0",inplace=True)
crypto_df.index.name=None
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [122]:
coin_name = crypto_df["CoinName"]
coin_name.head()

42        42 Coin
404       404Coin
1337    EliteCoin
BTC       Bitcoin
ETH      Ethereum
Name: CoinName, dtype: object

In [123]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns="CoinName",inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [124]:
# Use get_dummies() to create variables for text features.
crypto_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [125]:
X = pd.get_dummies(crypto_df,columns=["Algorithm","ProofType"])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
# Standardize the data with StandardScaler().
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


### Deliverable 2: Reducing Data Dimensions Using PCA

In [127]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
pca_results = pca.fit_transform(X_scaled)

In [128]:
# Create a DataFrame with the three principal components.
pca_df = pd.DataFrame(pca_results,index=names,columns=["PC 1","PC 2","PC 3"])
pca_df.index.name=None
pca_df.head()

,PC 1,PC 2,PC 3
42,-0.336814,1.018798,-0.546175
404,-0.320155,1.018979,-0.546547
1337,2.306786,1.673721,-0.583140
BTC,-0.153019,-1.287170,0.172142
ETH,-0.143733,-2.021654,0.328910


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [129]:
# Create an elbow curve to find the best value for K.
values = {}

for k in range(1,10):
    model=KMeans(n_clusters=k,random_state=1)
    model.fit(pca_df)
    values[k]= model.inertia_


c:\Users\David\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



In [130]:
values_df=pd.DataFrame.from_dict(values,orient="index",columns=["Intertia"])
values_df

,Intertia
1,3640.633436
2,2474.619606
3,1482.629654
4,547.610674
5,376.588982
6,262.902250
7,200.496745
8,155.048317
9,123.009421


In [131]:
values_df.hvplot(xlabel="k",title="Cryto Cluster Elbow Plot")

:Curve   [index]   (Intertia)

Running K-Means with `k=4`

In [132]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4,random_state=1)

# Fit and predict the model
groups = model.fit_predict(pca_df)

In [133]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same index.
clustered_df = crypto_df.merge(pca_df,left_index=True,right_index=True)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"]=coin_name

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"]=groups

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.336814,1.018798,-0.546175,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.320155,1.018979,-0.546547,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.306786,1.673721,-0.583140,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.153019,-1.287170,0.172142,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.143733,-2.021654,0.328910,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.165373,-1.143560,-0.048455,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.388816,1.240281,-0.373695,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.160957,-2.180871,0.406662,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.142175,-2.021752,0.328895,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.167379,-2.078323,0.360712,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [134]:
# Creating a 3D-Scatter with the PCA data and the clusters
px.scatter_3d(clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    hover_name="CoinName",
    hover_data=["Algorithm"])


In [135]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName","Algorithm","ProofType","TotalCoinSupply","TotalCoinsMined","Class"])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [136]:
# Print the total number of tradable cryptocurrencies.
print(len(clustered_df))

532


In [137]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
mms = MinMaxScaler()
scaled = mms.fit_transform(clustered_df[["TotalCoinSupply","TotalCoinsMined"]])

In [138]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(scaled,index=names,columns=["TotalCoinSupply","TotalCoinsMined"])
plot_df.index.name=None

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = clustered_df["Class"]

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [139]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined",y="TotalCoinSupply",by="Class",hover_cols="CoinName")


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)